In [8]:
from flair.nn import Classifier
from flair.data import Sentence

ner_tagger = Classifier.load('ner')
sentiment_tagger = Classifier.load('sentiment')

2025-03-27 19:50:51.912318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743076251.995724    1047 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743076252.022713    1047 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743076252.088048    1047 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743076252.088094    1047 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743076252.088100    1047 computation_placer.cc:177] computation placer alr

2025-03-27 19:51:02,789 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [2]:
from textblob import TextBlob
import math

In [54]:
from afinn import Afinn

afinn = Afinn()

In [56]:
sentence = "I am happy"
afinn.score(sentence)

3.0

In [78]:
import math

In [ ]:
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

book = 'worm'

ner_coref_data_dir = os.path.join('booknlp_output', book)
characters_data_dir = os.path.join('characters', book)


def collate_relations(
    ner_coref_data_dir: str,
    characters_data_dir: str
) -> None:
    main_characters_aliases_file_path = os.path.join(characters_data_dir, 'main_characters_aliases.json')

    if not os.path.exists(main_characters_aliases_file_path):
        raise FileNotFoundError('Missing main_characters_aliases.json in given directory!')

    with open(main_characters_aliases_file_path, 'r') as file: 
        main_char_list = json.load(file)

    num_chars = len(main_char_list)
    num_chapters = len(os.listdir(ner_coref_data_dir))

    relations_arr = np.zeros((num_chars, num_chars, 2, num_chapters))

    for chapter in tqdm(os.listdir(ner_coref_data_dir)):
        chapter_num = int(chapter.split('-')[1]) - 1
        relevant_sentences_file_path = os.path.join(ner_coref_data_dir, chapter, 'relevant_sentences.csv')
        df = pd.read_csv(relevant_sentences_file_path)

        df['Sentiment'] = 0.0

        context_sentiment = 0
        context_count = 0

        for idx, row in df.iterrows():
            sentiment = afinn.score(row['words'])
            # sentiment = math.sqrt(abs(sentiment)) * (-1 if sentiment < 0 else 1)
            df.loc[idx, 'Sentiment'] = sentiment

            context_sentiment += sentiment
            context_count += 1            

        # context_sentiment_avg = context_sentiment / context_count if context_count else 0
        # df['rolling_sentiment'] = df['Sentiment'].rolling(window=10, min_periods=1, center=True).mean()
        
        for idx, row in df.iterrows():
            char_list = set(json.loads(row['characters']) + (json.loads(row['speaker']) if type(row['speaker']) is str else []))
            if len(char_list) < 2:
                continue
        
            for char1 in char_list:
                for char2 in char_list:
                    if char1 == char2:
                        continue

                    relations_arr[char1][char2][0][chapter_num] += df.loc[idx, 'Sentiment']
                    relations_arr[char1][char2][1][chapter_num] += 1

        df.to_csv(relevant_sentences_file_path, index=False)

    character_relations_file_path = os.path.join(characters_data_dir, 'character-relations.npy')
    np.save(character_relations_file_path, relations_arr)

    print("Completed Sentiment Analysis!")


collate_relations(ner_coref_data_dir, characters_data_dir)

  0%|          | 0/304 [00:00<?, ?it/s]

100%|██████████| 304/304 [03:00<00:00,  1.68it/s]

Completed Sentiment Analysis!


In [6]:
main_characters_aliases_file_path = os.path.join(characters_data_dir, 'main_characters_aliases.json')

with open(main_characters_aliases_file_path, 'r') as file: 
    main_char_list = json.load(file)


In [236]:
fp = os.path.join(characters_data_dir, 'character-relations.npy')

arr = np.load(fp)

In [253]:
store = [[[0, 0] for j in range(len(main_char_list))]  for i in range(len(main_char_list))]

char_avgs = np.sum(arr, axis=(1, 3))

for i in range(len(main_char_list)):
    for j in range(len(main_char_list)):
        if i==j: 
            store[i][j] = None
            continue

        opposing_avg = char_avgs[j][0]/char_avgs[j][1]
        interaction_count = int(np.sum(arr[i][j][1]))

        store[i][j][0] = np.sum(arr[i][j][0]) / interaction_count - opposing_avg if interaction_count else 0
        store[i][j][1] = interaction_count

In [254]:
with open('interactions.json', 'w') as file:
    json.dump(store, file, indent=4)

In [250]:
focus_char = 5


focus_char_avg = char_avgs[focus_char][0]/char_avgs[focus_char][1]

for i in range(len(main_char_list)):
    opposing_avg = char_avgs[i][0]/char_avgs[i][1]

    print(i, main_char_list[i], np.sum(arr[focus_char][i][0]) / np.sum(arr[focus_char][i][1]) - opposing_avg)


0 ['NARRATOR', 'Taylor', 'Taylor Hebert', 'Ms. Hebert', 'Skitter', 'Weaver'] 0.21136947844256185
1 ['Tattletale', 'Lisa'] 0.5996662524861434
2 ['Grue', 'Brian'] 0.3829009336528134
3 ['Bitch', 'Rachel', 'Rachel Lindt'] 0.8238651807556403
4 ['Krouse', 'Francis', 'Trickster'] 0.37036404398419986
5 ['Coil', 'Thomas Calvert', 'Thomas', 'Calvert', 'Director Calvert', 'Commander Calvert'] nan
6 ['Lung', 'Kenta'] 0.4861294583883752
7 ['Noelle', 'Echidna'] 0.6453364147734706
8 ['Imp', 'Aisha'] 0.48904302755440376
9 ['Regent', 'Alec'] 1.1658628841607566
10 ['Jack', 'Jack Slash'] -2.9745476847592762
11 ['Miss Militia', 'Hannah', 'Hana'] 0.19495512764659761
12 ['Scion', 'the Warrior', 'The golden man', 'the golden man'] nan
13 ['Chevalier'] nan
14 ['Bonesaw', 'Riley'] -0.8950953678474114
15 ['Defiant', 'Armsmaster', 'Collin', 'Colin'] -1.4016337644656227
16 ['Amy', 'Amy Dallon', 'Panacea', 'Amelia', 'Ames'] 0.20331186752529898
17 ['Golem', 'Theo', 'Theodore Anders', 'Theodore'] nan
18 ['Weld'] -0.

/tmp/ipykernel_872/2179359057.py:9: RuntimeWarning: invalid value encountered in scalar divide
  print(i, main_char_list[i], np.sum(arr[focus_char][i][0]) / np.sum(arr[focus_char][i][1]) - opposing_avg)
